In [1]:
import copy
from ruamel.yaml import YAML
yaml=YAML(typ='safe')
import pandas as pd

from fup.profiles import profiles
from fup.utils.simulation_utils import get_start_values, run_simulations, overwrite_config

In [2]:
with open("config_template.yaml", "r") as f:
    config = yaml.load(f.read())

In [3]:
get_start_values(config=config)    

,name,class,info,income,expenses,value
0,crisis_2045,events.crisis.OilCrisis1973,start: 2045,NaN,NaN,NaN
1,events.crisis.OilCrisis1973,events.crisis.OilCrisis1973,start: None,NaN,NaN,NaN
2,main.environment.Inflation,main.environment.Inflation,inflation: 0.0,NaN,NaN,NaN
3,main.other.LuxuryExpenses,main.expenses.InflationSensitiveVariable,,0.0,2000.000000,NaN
4,main.other.OtherExpenses,main.expenses.InflationSensitive,,0.0,3000.000000,NaN
5,main.mobility.Car,main.expenses.InflationSensitive,,0.0,2000.000000,NaN
6,main.home.Flat,main.expenses.InflationSensitive,,0.0,6000.000000,NaN
7,main.work.Job,main.work.Job,,42000.0,0.000000,NaN
8,main.insurances.Pension,main.insurances.Pension,,0.0,3906.000000,NaN
9,main.insurances.Unemployment,main.insurances.Unemployment,,0.0,504.000000,NaN


In [4]:
scenarios = [
    dict(name="no luxury", config=dict(modules={"main.other.LuxuryExpenses": {'start_expenses': 0}})),
    dict(name="1k luxury", config=dict(modules={"main.other.LuxuryExpenses": {'start_expenses': 1000}})),
    dict(name="2k luxury", config=dict(modules={"main.other.LuxuryExpenses": {'start_expenses': 2000}})),
    dict(name="3k luxury", config=dict(modules={"main.other.LuxuryExpenses": {'start_expenses': 3000}})),
    dict(name="10k luxury", config=dict(modules={"main.other.LuxuryExpenses": {'start_expenses': 10000}})),
]

In [5]:
results = []
for scenario in scenarios:
    _config = copy.deepcopy(config)
    overwrite_config(_config, scenario["config"])
    
    df, df_stats = run_simulations(config=_config, runs=100)
   
    dfg = df.groupby("run").agg({"assets_cor": ["min", "max", "last"],
                             "expenses_net_cor": ["min", "max", "mean"]
                            })
    
    bankcruptcy_rate = (dfg["assets_cor"]["min"] <0).sum() / len(dfg)
    mean_savings_at_end = dfg["assets_cor"]["last"].mean()
    mean_expenses = dfg["expenses_net_cor"]["mean"].mean()
    
    results += [dict(name=scenario["name"],
                     config=_config,
                     bankcruptcy_rate=bankcruptcy_rate,
                     mean_savings_at_end=mean_savings_at_end,
                     mean_expenses=mean_expenses
                )]

In [6]:
df_results = pd.DataFrame(results)
df_results

,name,config,bankcruptcy_rate,mean_savings_at_end,mean_expenses
0,no luxury,"{'simulation': {'random': True, 'start_year': ...",0.00,193492.428280,11000.0
1,1k luxury,"{'simulation': {'random': True, 'start_year': ...",0.00,148846.747859,12000.0
2,2k luxury,"{'simulation': {'random': True, 'start_year': ...",0.03,133910.123847,13000.0
3,3k luxury,"{'simulation': {'random': True, 'start_year': ...",0.09,67456.237458,14000.0
4,10k luxury,"{'simulation': {'random': True, 'start_year': ...",1.00,-519809.010996,21000.0
